In [1]:
import torch
from torchvision import utils

import math
import os

from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Sampler, tensor_info, histc

from matplotlib import pyplot as plt

import numpy as np

/home/ryuuyou/.conda/envs/dm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model

In [2]:
sampling_timesteps = 1000
is_ddim_sampling = False

# sampling_timesteps = 250
# is_ddim_sampling = True

In [3]:
model = Unet(
    dim = 64,
    channels=1,
    dim_mults = (1, 2, 4, 8)
)
diffusion = GaussianDiffusion(
    model,
    image_size = 128,
    timesteps = 1000,
    sampling_timesteps = sampling_timesteps,
    loss_type = 'l1',
    is_ddim_sampling=is_ddim_sampling
)
sampler = Sampler(diffusion)

In [4]:
base_path = 'pretrained_y/'
model_type = ['no_normalize', 'z_score', 'min_max'][1]
model_path = os.path.join(base_path, model_type, 'model-20.pt')
print(f'model path: {model_path}')

sampler.load(path=model_path)

model path: pretrained_y/z_score/model-20.pt
loading from: [version]:1.5.4; [step]:200000


# Sample

In [5]:
num_samples = 16
return_all_timesteps = False

In [6]:
res = sampler.sample(num_samples=num_samples, return_all_timesteps=return_all_timesteps)
tensor_info(res)

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step: 100%|██████████| 1000/1000 [01:15<00:00, 13.17it/s]


shape: torch.Size([16, 1, 128, 128]) 
type: torch.float32 
max: 1.0           
min: 0.0 
mean: 0.4393215477466583 
std: 0.3429486155509949


# Save

In [7]:
save_folder = os.path.join('sample_out/', model_type)

In [8]:
if not return_all_timesteps:
    sampler.save_tif(res, folder=save_folder)
else:
    sampler.save_tif_with_records(res, folder=os.path.join(save_folder, 'record/'), file_name='record_one_batch.tif', step=10)